In [1]:
from functools import partial
import os

import torch.nn as nn
from torchvision.models import vit_b_16
import torch

import argparse

from external.dfr.wb_data import WaterBirdsDataset, get_loader, get_transform_cub, log_data
from external.dfr.utils import evaluate, get_y_p

from external.dfr.wb_data import WaterBirdsDataset, get_loader, get_transform_cub, log_data
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
import pickle



In [2]:
NUM_CLASSES = 2


In [3]:
model = vit_b_16(weights='DEFAULT')

model.heads.head = nn.Linear(model.heads.head.in_features, NUM_CLASSES)


In [4]:
model

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

In [5]:
for name, param in model.named_parameters():
    print(name, param.requires_grad)

class_token True
conv_proj.weight True
conv_proj.bias True
encoder.pos_embedding True
encoder.layers.encoder_layer_0.ln_1.weight True
encoder.layers.encoder_layer_0.ln_1.bias True
encoder.layers.encoder_layer_0.self_attention.in_proj_weight True
encoder.layers.encoder_layer_0.self_attention.in_proj_bias True
encoder.layers.encoder_layer_0.self_attention.out_proj.weight True
encoder.layers.encoder_layer_0.self_attention.out_proj.bias True
encoder.layers.encoder_layer_0.ln_2.weight True
encoder.layers.encoder_layer_0.ln_2.bias True
encoder.layers.encoder_layer_0.mlp.0.weight True
encoder.layers.encoder_layer_0.mlp.0.bias True
encoder.layers.encoder_layer_0.mlp.3.weight True
encoder.layers.encoder_layer_0.mlp.3.bias True
encoder.layers.encoder_layer_1.ln_1.weight True
encoder.layers.encoder_layer_1.ln_1.bias True
encoder.layers.encoder_layer_1.self_attention.in_proj_weight True
encoder.layers.encoder_layer_1.self_attention.in_proj_bias True
encoder.layers.encoder_layer_1.self_attention.ou

In [6]:
for param in model.parameters():
    param.requires_grad = False

for param in model.heads.head.parameters():
    param.requires_grad = True

#for name, module in model.named_modules():
#    if "mlp" in name and isinstance(module, torch.nn.Linear):
#        for param in module.parameters():
#            param.requires_grad = True

 
#for name, module in model.named_modules():
#    if "encoder.ln" in name and isinstance(module, torch.nn.LayerNorm):
#        for param in module.parameters():
#            param.requires_grad = True
            
for name, param in model.named_parameters():
    if param.requires_grad:
            print(f"{name}: {param.requires_grad}")
    

heads.head.weight: True
heads.head.bias: True


In [7]:
model_orig = vit_b_16(weights='DEFAULT')
model_orig.heads.head = nn.Linear(model_orig.heads.head.in_features, NUM_CLASSES)
model_orig.load_state_dict(torch.load(
    "logs/vit_waterbirds.pth",map_location=torch.device('cpu')
    ))

C:\Users\elmop\AppData\Local\Temp\ipykernel_2436\694165624.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_orig.load_state_dict(torch.load(


FileNotFoundError: [Errno 2] No such file or directory: 'logs/vit_waterbirds.pth'

In [ ]:
model2 = vit_b_16(weights='DEFAULT')
model2.heads.head = nn.Linear(model2.heads.head.in_features, NUM_CLASSES)
model2.load_state_dict(torch.load(
    "logs/dfr_model.pth",map_location=torch.device('cpu')
    ))

In [ ]:
def get_average_weights(weights):
    return np.mean(np.abs(weights), axis=0)

In [ ]:
pre_weights = get_average_weights(model_orig.heads.head.weight.data.cpu().numpy()).reshape(24,32)
post_weights = get_average_weights(model2.heads.head.weight.data.cpu().numpy()).reshape(24,32)



In [ ]:
get_average_weights(model_orig.heads.head.weight.data.cpu().numpy())

In [ ]:

with open('logs/dfr_logreg.pth', 'rb') as file:
    logreg = pickle.load(file)
    
logreg.coef_

In [ ]:
logreg_weights = np.abs(logreg.coef_)

In [ ]:
def plot_weight_grids(avg_before_reshaped, avg_after_reshaped, cmap='CMRmap',title1 = "vit_waterbird", title2 = "dfr"):
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    vmin = min(avg_before_reshaped.min(), avg_after_reshaped.min())
    vmax = min(avg_before_reshaped.max(), avg_after_reshaped.max())##set to max to change colour scale

    im1 = axes[0].imshow(avg_before_reshaped, cmap=cmap, aspect='auto', vmin=vmin, vmax=vmax)
    axes[0].set_title(title1)
    axes[0].set_xlabel('')
    axes[0].set_ylabel('')
    plt.colorbar(im1, ax=axes[0])
    
    im2 = axes[1].imshow(avg_after_reshaped, cmap=cmap, aspect='auto', vmin=vmin, vmax=vmax)
    axes[1].set_title(title2)
    axes[1].set_xlabel('')
    axes[1].set_ylabel('')
    plt.colorbar(im2, ax=axes[1])

    plt.tight_layout()
    plt.show()




**last layer retrained**

In [ ]:
plot_weight_grids(pre_weights, post_weights, title2="dfr_last_layer_retrained")

**logreg**

In [ ]:
plot_weight_grids(pre_weights, logreg_weights.reshape(24,32), title2="dfr_last_layer_logreg")


In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
nr_pre = scaler.fit_transform(pre_weights)
nr_post = scaler.fit_transform(post_weights)
nr_logreg = scaler.fit_transform(logreg_weights.reshape(24,32))

**Normalised**

In [ ]:
plot_weight_grids(nr_logreg, nr_post, title1="nr_logreg", title2="nr_dfr_finetune") ## testing how they look normalised


In [ ]:
np.max(nr_post)

In [ ]:
model2.heads.head.weight.data.cpu().numpy().shape


In [ ]:
one = np.abs(model2.heads.head.weight.data.cpu().numpy()[0,:]).reshape(24,32)
two = np.abs(model2.heads.head.weight.data.cpu().numpy()[1,:]).reshape(24,32)



**W1 vs W2**

In [ ]:
plot_weight_grids(one, two)